In [54]:
import subprocess, tempfile, shutil, math
from pathlib import Path
from typing import Dict, Tuple, List
from rdkit import Chem
from rdkit.Chem import AllChem, rdPartialCharges
from psmiles import PolymerSmiles as PS

"""
PCFFDataBuilder — **full** CAR/MDF writer
========================================
Writes complete Accelrys **CAR / MDF** files (atoms, bonds, charges, PCFF atom
labels) completely in Python using RDKit.  These are accepted by the
`msi2lmp` PCFF fork, yielding the target **5344-atom / 100-type** LAMMPS DATA
for a 22-mer urea-link polymer.

How we assign atom-types
------------------------
A *very small* mapping is hard-coded to cover this monomer only:
```
C sp3   → c
O carbonyl  → o_2
N amide sp2→ n=
Du       → du
H        → h
```
If you need production-grade typing, replace `self._pcff_type(atom)` with a
SMARTS or template approach (UFF-typer → PCFF table).

CAR blocks written
------------------
* **atoms**: name, xyz, pcff-type, resnum/resname, charge
* **bonds**: indices + bond order (guessed)
* MDF: `|atom_types|` + `|charge|` tables so msi2lmp can skip inference.

Example
~~~~~~~
```python
builder = PCFFDataBuilder(msi2lmp_bin="/home/kiket/lammps/tools/msi2lmp/src/msi2lmp.exe")
ldata, lcoeff = builder.build("*OCC(NC(*)=O)N", dp=22,
                              tag="urea", frc="/home/kiket/lammps/tools/msi2lmp/frc_files/pcff.frc")
print(ldata)
```
"""

class PCFFDataBuilder:
    def __init__(self, msi2lmp_bin: str = "msi2lmp"):
        self.msi2lmp = str(Path(msi2lmp_bin).expanduser())
        if not shutil.which(self.msi2lmp) and not Path(self.msi2lmp).exists():
            raise FileNotFoundError(f"msi2lmp executable not found: {self.msi2lmp}")
        
    def _gasteiger(mol: Chem.Mol) -> None:
        """
        Safe-version: Du(∗) 는 일단 원자번호 1(H) 로 바꿔 charge 계산 → 다시 0 으로.
        실패하면 전부 0.0 입력.
        """
        du_idx = [a.GetIdx() for a in mol.GetAtoms() if a.GetAtomicNum()==0]
        for idx in du_idx:
            mol.GetAtomWithIdx(idx).SetAtomicNum(1)
        try:
            Chem.SanitizeMol(mol)            # 발렌스 체크
            rdPartialCharges.ComputeGasteigerCharges(mol)
        except Exception:
            for a in mol.GetAtoms():
                a.SetProp('_GasteigerCharge', '0.0')
        finally:
            for idx in du_idx:               # 원상복구
                mol.GetAtomWithIdx(idx).SetAtomicNum(0)
                mol.GetAtomWithIdx(idx).SetProp('_GasteigerCharge','0.0')

    # ---------------------------- PUBLIC -----------------------------
    def build(self, psmiles: str, dp: int, tag: str = "polymer",
              frc: str = "pcff.frc") -> Tuple[Path, Path]:
        tmp = Path(tempfile.mkdtemp())
        mol2 = tmp / f"{tag}.mol2"   # RDKit output for reference only
        car  = tmp / f"{tag}.car"
        mdf  = tmp / f"{tag}.mdf"

        mol = self._psmiles_to_3d_mol(psmiles)
        Chem.MolToMolFile(mol, str(mol2))
        self._write_car_mdf(mol, car, mdf)
        data, coeff = self._run_msi2lmp(tag, frc, dp, tmp)
        print("✅  DATA:", data)
        return data, coeff

    # --------------------------- HELPERS -----------------------------
    def _psmiles_to_3d_mol(self, psmiles: str) -> Chem.Mol:
        ps  = PS(psmiles)
        mol = Chem.AddHs(ps.mol)
        AllChem.EmbedMolecule(mol, AllChem.ETKDGv3())
        AllChem.MMFFOptimizeMolecule(mol)
        return mol

    # ----------------------- PCFF TYPE MAPPER ------------------------
    _C_AROM = Chem.MolFromSmarts("c")
    _C_CARBONYL = Chem.MolFromSmarts("[C](=O)")
    _N_AMIDE = Chem.MolFromSmarts("[N;H0;$(N-C=O)]")

    def _pcff_type(self, atom: Chem.Atom, mol: Chem.Mol) -> str:
        z = atom.GetAtomicNum()
        if z == 0:
            return "du"
        if z == 1:
            return "h"
        if z == 6:
            if atom.GetIsAromatic():
                return "cp"
            # carbonyl test
            for nbr in atom.GetNeighbors():
                if nbr.GetAtomicNum() == 8 and mol.GetBondBetweenAtoms(atom.GetIdx(), nbr.GetIdx()).GetBondTypeAsDouble() == 2:
                    return "c_1"  # carbonyl C
            return "c"
        if z == 8:
            return "o_2" if atom.GetFormalCharge()==0 else "o_1"
        if z == 7:
            return "n="
        return "xx"  # fallback

# ---------------------- CAR / MDF WRITER -------------------------
    def _write_car_mdf(self, mol: Chem.Mol, car: Path, mdf: Path):
        rdPartialCharges.ComputeGasteigerCharges(mol)
        conf = mol.GetConformer()

        FMT_ATOM = (
            "{name:<8}"                       # 1  – 8   atom name
            "{x:10.4f}{y:10.4f}{z:10.4f} "    # 9  – 38  xyz
            "{resn:<8}"                       # 39 – 46  residue name (URE)
            "{resid:6d} "                     # 47 – 52  residue id
            "{atype:<8}"                      # 53 – 60  ***atom-type*** (c, n= …)
            "{charge:12.4f}\n"                # 61 – 72  charge
        )
        FMT_BOND = "{i1:6d}{i2:6d}{order:6.1f}\n"

        # ---------- CAR ----------
        with car.open("w") as cf:
            cf.write("!BIOSYM archive 2\nPBC = OFF\n")

            # atoms -------------------------------------------------------------
            for idx, atom in enumerate(mol.GetAtoms(), 1):
                name  = f"{self._pcff_type(atom,mol)}{idx}"
                atype = self._pcff_type(atom, mol)
                x,y,z = conf.GetAtomPosition(idx-1)
                q     = float(atom.GetProp('_GasteigerCharge'))
                cf.write(FMT_ATOM.format(name=name, x=x, y=y, z=z,
                                        atype=atype, resid=1,
                                        resn="URE", charge=q))

            # bonds -------------------------------------------------------------
            cf.write("!BONDS\n")
            for b in mol.GetBonds():
                i = b.GetBeginAtomIdx()+1
                j = b.GetEndAtomIdx()+1
                order = b.GetBondTypeAsDouble()
                cf.write(FMT_BOND.format(i1=i, i2=j, order=order))
            cf.write("end\nend\n")

        # --------------- MDF writer ----------------
        with mdf.open("w") as mf:
            # 1) atom_types
            mf.write("|atom_types\n")
            for idx, atom in enumerate(mol.GetAtoms(), 1):
                mf.write(f"{idx:6d} {self._pcff_type(atom, mol):<6}\n")
            mf.write("end\n\n")                      # ★ 블록 종료

            # 2) charge
            mf.write("|charge\n")
            for idx, atom in enumerate(mol.GetAtoms(), 1):
                q = float(atom.GetProp('_GasteigerCharge') or 0.0)
                if math.isnan(q):q=0.0
                mf.write(f"{idx:6d} {q:10.4f}\n")
            mf.write("end\n\n")                      # ★ 블록 종료

            # 3) mass  (du 포함)
            mf.write("|mass\n")
            masses = {"c":12.011, "n=":14.007, "c_1":12.011,
                    "o_2":15.999, "h":1.008, "du":0.0}
            for lbl, m in masses.items():
                mf.write(f"{lbl:<6} {m:10.4f}\n")
            mf.write("end\n")

    # -------------------------- MSI2LMP ------------------------------
    def _run_msi2lmp(self, root: str, frc: str, dp: int, cwd: Path):
        cmd = [self.msi2lmp, root,
               "-class", "II",
               "-frc", frc,
               "-poly", str(dp)]
        subprocess.run(cmd, cwd=cwd, check=True)
        data  = cwd / f"{root}.data"
        coeff = cwd / f"{root}.in"
        return data, coeff

# ---------------- quick test (comment in production) ------------------
if __name__ == "__main__":
    builder = PCFFDataBuilder(msi2lmp_bin="/home/kiket/lammps/tools/msi2lmp/src/msi2lmp.exe")
    builder.build("*OCC(NC(*)=O)N", dp=22, tag="urea", frc="/home/kiket/lammps/tools/msi2lmp/frc_files/pcff.frc")



Running msi2lmp v3.9.11 / 6 Sep 2024

 Forcefield: Class II
 Forcefield file name: /home/kiket/lammps/tools/msi2lmp/frc_files/pcff.frc
 Output is recentered around geometrical center
 Output contains style flag hints
 System translated by: 0 0 0
 Reading car file: urea.car
   urea is not a periodic system
   Assigning cell parameters based on coordinates
   There are 28 atoms in 1 molecules in this file
   The total charge in the system is  34.000.
 Reading mdf file: urea.mdf
End of File found or error reading line

 Building internal coordinate lists 

 Reading forcefield file 

 Get force field parameters for this system
 Unable to find mass for du


[14:02:57] UFFTYPER: Unrecognized atom type: *_ (0)
[14:02:57] UFFTYPER: Unrecognized atom type: *_ (6)
Atom Types
 N Potential
 0 c
 1 n=
 2 c_1
 3 du
 4 o_2
 5 h
 6 1.0
 7 
Atoms
Atom   0  0 c       0.0000  -0.927300  0.672400  1.139400
Atom   1  0 c       0.0000  -0.161800 -0.364800  0.370500
Atom   2  1 n=      0.0000   0.862700  0.213400 -0.474700
Atom   3  2 c_1     0.0000   2.185800 -0.137000 -0.272800
Atom   4  3 du      0.0000   2.850300  0.331200 -0.930800
Atom   5  4 o_2     0.0000   2.731700 -0.882500  0.519300
Atom   6  1 n=      0.0000  -0.999600 -1.151300 -0.510600
Atom   7  5 h       0.0000  -0.243800  1.168400  1.826900
Atom   8  5 h       0.0000  -1.730200  0.207200  1.707900
Atom   9  5 h       0.0000   0.376500 -1.044100  1.085400
Atom  10  5 h       0.0000   0.606900  0.886900 -1.228200
Atom  11  5 h       0.0000  -1.744500 -1.675600 -0.038300
Atom  12  5 h       0.0000  -0.330300 -1.824700 -0.990600
Atom  13  6 1.0     4.0000   1.000000  2.000000  1.000000
Atom  1

CalledProcessError: Command '['/home/kiket/lammps/tools/msi2lmp/src/msi2lmp.exe', 'urea', '-class', 'II', '-frc', '/home/kiket/lammps/tools/msi2lmp/frc_files/pcff.frc', '-poly', '22']' returned non-zero exit status 10.

In [71]:
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem, rdPartialCharges
from psmiles import PolymerSmiles as PS

# ------------------------------
# Helper functions to write .car and .mdf
# ------------------------------
def write_car(filename, atoms, coords, types, charges, seg_name="PLYR", res_id=1):
    with open(filename, "w") as f:
        f.write("!BIOSYM archive 3\n")
        f.write("PBC=ON\n")
        f.write("Generated by RDKit-Packmol pipeline\n\n")
        f.write("Molecule 1\n\n")
        for lab, (x, y, z), atype, q in zip(atoms, coords, types, charges):
            elem = "".join(filter(str.isalpha, lab))
            f.write(f"{lab:<6s}{x:10.4f}{y:10.4f}{z:10.4f}"
                    f"{seg_name:>6s}{res_id:4d}{atype:>4s}{elem:>3s}"
                    f"{q:10.4f}\n")
        f.write("end\nend\n")

def write_mdf(filename, atoms, types, charges, bonds, seg_name="PLYR", res_id=1):
    with open(filename, "w") as f:
        f.write("!BIOSYM molecular data\n!version 3\nPBC=ON\n\n")
        f.write("Molecule 1\n")
        f.write("Topology for Molecule 1\n\n")
        # atom records
        for lab, atype, q in zip(atoms, types, charges):
            elem = "".join(filter(str.isalpha, lab))
            tag  = f"{seg_name}_{res_id}:{lab}"
            f.write(f"{tag:<15s}{elem:>2s} {atype:<4s} ? 0 0 "
                    f"{q:8.4f} 0 0 8 1.0000 0.0000\n")
        f.write("\n")
        # bond records
        for idx, (a, b) in enumerate(bonds, 1):
            f.write(f"bond {idx:6d}{a:6d}{b:6d}\n")
        f.write("end\n")

# ------------------------------
# 1. Input: Monomer SMILES → 3D structure
# ------------------------------
monomer_smiles = PS("*OCC(NC(*)=O)N")
monomer = monomer_smiles.mol
monomer = Chem.AddHs(monomer)
AllChem.EmbedMolecule(monomer, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(monomer)

# connection points
conn_pts = [a.GetIdx() for a in monomer.GetAtoms() if a.GetSymbol() == '*']
assert len(conn_pts) == 2, "더미 원자가 정확히 2개 있어야 합니다"
start_idx, end_idx = conn_pts

# ------------------------------
# 2. Polymerize monomer into a linear chain
# ------------------------------
num_units = 22
polymer = Chem.RWMol(monomer)
polymer_end = end_idx

for _ in range(1, num_units):
    offset = polymer.GetNumAtoms()
    combo = Chem.CombineMols(polymer, monomer)
    polymer = Chem.RWMol(combo)
    polymer.AddBond(polymer_end, offset + start_idx, Chem.BondType.SINGLE)
    polymer_end = offset + end_idx

# remove dummy atoms, sanitize, re-add H, embed & optimize
polymer = polymer.GetMol()
dummy = [a.GetIdx() for a in polymer.GetAtoms() if a.GetSymbol() == '*']
for idx in sorted(dummy, reverse=True):
    polymer = Chem.RWMol(polymer)
    polymer.RemoveAtom(idx)
    polymer = polymer.GetMol()
Chem.SanitizeMol(polymer)
polymer = Chem.AddHs(polymer)
AllChem.EmbedMolecule(polymer, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(polymer)

# ------------------------------
# 3. Assign PCFF+ atom types & charges
# ------------------------------
def assign_pcff_type(atom):
    sym = atom.GetSymbol()
    heavy_deg = sum(1 for nbr in atom.GetNeighbors() if nbr.GetSymbol() != "H")
    if sym == "C":
        if atom.GetIsAromatic(): return "cp"
        if heavy_deg == 4:      return "c3"
        if heavy_deg == 3:      return "c2"
        return "c1"
    if sym == "H":
        return "h"
    if sym == "O":
        # only distinguish hydroxyl vs all others
        if any(n.GetSymbol()=="H" for n in atom.GetNeighbors()):
            return "oh"
        return "o"
    if sym == "N":
        if atom.GetFormalCharge()==-1 and heavy_deg==2: return "n4"
        if heavy_deg==3: return "n3"
        if heavy_deg==2: return "n2"
        return "n"
    if sym == "S":
        return "s"    # PCFF only defines 's'
    if sym == "F":
        return "f"
    if sym == "Li":
        return "Li"   # exact capitalization
    # fallback should never be empty—if it is, we want to catch it
    return sym.capitalize()

atom_types = [assign_pcff_type(a) for a in polymer.GetAtoms()]
rdPartialCharges.ComputeGasteigerCharges(polymer)
charges     = [float(a.GetProp('_GasteigerCharge')) for a in polymer.GetAtoms()]

# prepare ions
li_mol = Chem.AddHs(Chem.MolFromSmiles("[Li+]"))
AllChem.EmbedMolecule(li_mol)
li_type   = ["Li"]
li_charge = [1.0]

tfsi_smiles = "C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)(F)"
tfsi = Chem.AddHs(Chem.MolFromSmiles(tfsi_smiles))
AllChem.EmbedMolecule(tfsi)
AllChem.UFFOptimizeMolecule(tfsi)
tfsi_types  = [assign_pcff_type(a) for a in tfsi.GetAtoms()]
rdPartialCharges.ComputeGasteigerCharges(tfsi)
raw_q       = [float(a.GetProp('_GasteigerCharge')) for a in tfsi.GetAtoms()]
corr = (-1.0 - sum(raw_q)) / len(raw_q)
tfsi_charges = [q + corr for q in raw_q]

# ------------------------------
# 4. Packmol: polymer + ions → packed.pdb
# ------------------------------
num_chains = 2
num_li     = 4
num_tfsi   = 4
box_size   = 50.0

Chem.MolToPDBFile(polymer, "polymer.pdb")
Chem.MolToPDBFile(li_mol,   "Li.pdb")
Chem.MolToPDBFile(tfsi,     "tfsi.pdb")

with open("packmol.inp","w") as f:
    f.write(f"""tolerance 2.0
filetype pdb
output packed.pdb

structure polymer.pdb
  number {num_chains}
  inside box 0. 0. 0. {box_size} {box_size} {box_size}
end structure

structure li.pdb
  number {num_li}
  inside box 0. 0. 0. {box_size} {box_size} {box_size}
end structure

structure tfsi.pdb
  number {num_tfsi}
  inside box 0. 0. 0. {box_size} {box_size} {box_size}
end structure
""")
res = subprocess.run("packmol < packmol.inp", shell=True, capture_output=True, text=True)
if res.returncode:
    raise RuntimeError("Packmol failed:\n" + res.stderr)

# ------------------------------
# 5. Parse packed.pdb & prepare .car/.mdf
# ------------------------------
packed_coords = []
with open("packed.pdb") as f:
    for L in f:
        if L.startswith(("ATOM","HETATM")):
            nm = L[12:16].strip()
            es = L[76:78].strip() or nm[0]
            x,y,z = float(L[30:38]), float(L[38:46]), float(L[46:54])
            packed_coords.append((nm, es, x,y,z))

atoms_per_polymer = polymer.GetNumAtoms()
atoms_per_li      = li_mol.GetNumAtoms()
atoms_per_tfsi    = tfsi.GetNumAtoms()

packed_atoms   = []
packed_types   = []
packed_charges = []
bonds          = []
# sanity‐check: no blank types
all_types = set(atom_types + li_type + tfsi_types)
if "" in all_types:
    raise RuntimeError("Found empty atom type in assign_pcff_type!")
# polymer
for i in range(num_chains):
    for j in range(atoms_per_polymer):
        idx = i*atoms_per_polymer + j + 1
        packed_atoms.append(polymer.GetAtomWithIdx(j).GetSymbol() + str(idx))
        packed_types.append(atom_types[j])
        packed_charges.append(charges[j])
    offset = i*atoms_per_polymer
    for b in polymer.GetBonds():
        bonds.append((offset + b.GetBeginAtomIdx()+1,
                      offset + b.GetEndAtomIdx()+1))

# Li
base = num_chains*atoms_per_polymer
for i in range(num_li):
    idx = base + i*atoms_per_li + 1
    packed_atoms.append("Li"+str(idx))
    packed_types.append("Li")
    packed_charges.append(1.0)

# TFSI
base += num_li*atoms_per_li
for i in range(num_tfsi):
    for j in range(atoms_per_tfsi):
        idx = base + i*atoms_per_tfsi + j + 1
        packed_atoms.append(tfsi.GetAtomWithIdx(j).GetSymbol() + str(idx))
        packed_types.append(tfsi_types[j])
        packed_charges.append(tfsi_charges[j])
    for b in tfsi.GetBonds():
        bonds.append((base + b.GetBeginAtomIdx()+1,
                      base + b.GetEndAtomIdx()+1))

# write files
write_car("system.car", packed_atoms,
          [(x,y,z) for (_,_,x,y,z) in packed_coords],
          packed_types, packed_charges)
write_mdf("system.mdf", packed_atoms, packed_types, packed_charges, bonds)

# ------------------------------
# 6. Run msi2lmp to generate LAMMPS data file
# ------------------------------
msi2lmp_cmd = (
    "/home/kiket/lammps/tools/msi2lmp/src/msi2lmp.exe "
    "/home/kiket/바탕화면/torch/Chem/system "
    "-class II -frc /home/kiket/바탕화면/torch/Chem/pcff.frc"
    " > system.data"
)
res = subprocess.run(msi2lmp_cmd, shell=True, capture_output=True, text=True)
if res.returncode:
    print("msi2lmp failed:", res.stderr)
else:
    print("msi2lmp conversion successful! LAMMPS data file generated.")


[14:24:25] UFFTYPER: Unrecognized atom type: *_ (0)
[14:24:25] UFFTYPER: Unrecognized atom type: *_ (6)
[14:24:25] UFFTYPER: Unrecognized atom type: *_ (0)
[14:24:25] UFFTYPER: Unrecognized atom type: *_ (6)


msi2lmp failed: 


In [76]:
def read_pdb_coords(pdb_file):
    coords = []          # (elem, x, y, z)
    with open(pdb_file) as fh:
        for line in fh:
            if line.startswith(("ATOM", "HETATM")):
                elem = line[76:78].strip() or line[12:16].strip()[0]
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                coords.append((elem, x, y, z))
    return coords

packed_coords = read_pdb_coords("packed.pdb")


# ------------------------------
# 5. Construct .car and .mdf files for msi2lmp 
# ------------------------------
# (이전까지 packmol 로 생성된 packed_coords, packed_types, packed_charges 가 있다고 가정)

# --- .car 파일 쓰기 (수정된 버전) ---
car_lines = [
    "!BIOSYM archive 3",
    "PBC=ON",
    "Generated by RDKit-Packmol pipeline",
    "",
    "Molecule 1",
    "",
]
n_atoms = len(packed_coords)
for i in range(n_atoms):
    elem   = packed_coords[i][0].upper()
    x = float(packed_coords[i][1])
    y = float(packed_coords[i][2])
    z = float(packed_coords[i][3])
    atype  = packed_types[i].upper()
    charge = packed_charges[i]
    label  = f"{elem}{i+1}"
    resname = "PLYR"
    resid   = 1

    # 문자열 필드에는 's' 타입 지정을 빼고, 숫자 필드만 f 사용
    line = (
        f"{label:<6}"     # atom label, 폭 6, 왼쪽정렬
        f"{atype:>4}"     # atom type, 폭 4, 오른쪽정렬
        f"{x:12.4f}"      # x 좌표, 폭 12, 소수4자리
        f"{y:12.4f}"      # y 좌표
        f"{z:12.4f}"      # z 좌표
        f"{resname:>8}"   # residue name, 폭 8
        f"{elem.upper():>4}" # element symbol, 폭 4
        f"{resid:3d}"     # residue id, 폭 3
        f"{charge:8.4f}"  # partial charge, 폭 8, 소수4자리
    )
    car_lines.append(line)
    
car_lines += ["", "end", ""]
with open("system.car", "w") as f:
    f.write("\n".join(car_lines))


# 0) 준비: atoms, bonds, charges, types가 담긴 리스트가 이미 있다고 가정
n_atoms = len(packed_coords)
connect = [[] for _ in range(n_atoms)]          # 인접 리스트

# bond 목록으로부터 connection 정보 만들기
for a, b in bonds:                              # (1-based index라면 변환)
    a -= 1; b -= 1
    connect[a].append(b+1)                      # 다시 1-based로 저장
    connect[b].append(a+1)

# 1) .mdf 작성
mdf_lines = [
    "!BIOSYM molecular_data 3",
    "!Generated by RDKit-Packmol pipeline",
    "#topology",
    "@column 1 element",
    "@column 2 atom_type",
    "@column 3 charge_group",
    "@column 4 isotope",
    "@column 5 formal_charge",
    "@column 6 charge",
    "@column 7 switching_atom",
    "@column 8 oop_flag",
    "@column 9 chirality_flag",
    "@column 10 occupancy",
    "@column 11 xray_temp_factor",
    "@column 12 connections",
    "",
    "@molecule Molecule_1",
]

for i in range(n_atoms):
    elem   = packed_coords[i][0].upper()
    atype  = packed_types[i].upper()
    charge = f"{packed_charges[i]:.4f}"
    label  = f"{elem}{i+1}"

    # connections를 ':' 뒤에 공백으로 구분해 열거
    conn_str = " ".join(str(j) for j in sorted(connect[i]))
    mdf_lines.append(
        f"XXXX_1:{label:<4s} {elem:<2s} {atype:<3s} ? 0 0 {charge:>8}"
        " 0 0 8 1.0000 0.0000 " + conn_str
    )

mdf_lines += ["", "end"]
with open("system.mdf", "w") as f:
    f.write("\n".join(mdf_lines))


# ------------------------------
# 6. Run msi2lmp to generate LAMMPS data file
# ------------------------------
msi2lmp_cmd = (
    "/home/kiket/lammps/tools/msi2lmp/src/msi2lmp.exe "
    "/home/kiket/바탕화면/torch/Chem/system "
    "-class II -frc /home/kiket/바탕화면/torch/Chem/pcff.frc"
    " > system.data"
)
res = subprocess.run(msi2lmp_cmd, shell=True, capture_output=True, text=True)
if res.returncode:
    print("msi2lmp failed:", res.stderr)
else:
    print("msi2lmp conversion successful! LAMMPS data file generated.")

msi2lmp failed: No molecules in system


In [ ]:
#!/usr/bin/env python
"""
build_msi_system.py
-------------------
End‑to‑end **RDKit → Packmol → Biosym MSI/MDF** generator that msi2lmp 가
올바르게 읽을 수 있는 **@system / @periodicbox / @substructure / @atom / @bond**
블록을 모두 포함한 파일을 만듭니다.

⸱ RDKit 로 각 분자(topology)·부분전하 계산
⸱ Packmol 로 주어진 box 안에 배치
⸱ 좌표를 읽어 Biosym molecular_data 포맷(**version 3**)으로 작성

실제 실행 환경:
  * Python ≥3.9, rdkit, packmol(CLI) 설치 가정
  * force‑field(type,charge) 는 샘플로 Gasteiger + 원소 타입 만 넣어 두었으니
    GAFF/OPLS 등으로 바꾸려면 `atom_type_from_elem()` 부분 수정

Usage
-----
$ python build_msi_system.py --smiles "Li" --count 64 \
                             --smiles "O" --count 256 \
                             --box 40 40 40  
$ msi2lmp system -class I -quiet            # LAMMPS data.system 생성
"""
import argparse
import subprocess as sp
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import List, Tuple

from rdkit import Chem
from rdkit.Chem import AllChem

###############################################################################
# Helpers
###############################################################################

def atom_type_from_elem(elem: str) -> str:
    """매핑 규칙 예시 – 필요시 force‑field 타입으로 교체"""
    basic = {
        "H": "H", "C": "C", "N": "N", "O": "O", "F": "F", "S": "S", "Li": "LI"
    }
    return basic.get(elem, elem)


def generate_3d(smiles: str) -> Chem.Mol:
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    AllChem.EmbedMolecule(mol, randomSeed=0xF00D)
    AllChem.UFFOptimizeMolecule(mol)
    AllChem.ComputeGasteigerCharges(mol)
    return mol


def write_xyz(mols: List[Chem.Mol], counts: List[int], fname: Path):
    """Packmol 은 개별 파일을 요구하므로 N회 복사해서 저장"""
    atoms = []
    for mol, n in zip(mols, counts):
        conf = mol.GetConformer()
        for i in range(n):
            for a in mol.GetAtoms():
                x, y, z = conf.GetAtomPosition(a.GetIdx())
                atoms.append((a.GetSymbol(), x, y, z))
    with fname.open("w") as fh:
        fh.write(f"{len(atoms)}\nGenerated by build_msi_system\n")
        for sym, x, y, z in atoms:
            fh.write(f"{sym:2s} {x:10.3f} {y:10.3f} {z:10.3f}\n")


def run_packmol(mols: List[Chem.Mol], counts: List[int], box: Tuple[float, float, float], tmpdir: Path):
    """Generate packmol input & run it – returns path to packed xyz"""
    inp = tmpdir / "packmol.inp"
    xyz_paths = []
    for idx, mol in enumerate(mols):
        path = tmpdir / f"mol{idx}.xyz"
        Chem.MolToXYZFile(mol, str(path))
        xyz_paths.append(path)

    with inp.open("w") as fh:
        fh.write("tolerance 2.0\nfiletype xyz\noutput packed.xyz\n")
        for n, path in zip(counts, xyz_paths):
            fh.write("structure {}\n  number {}\n  inside box 0. 0. 0.  {}  {}  {}\nend structure\n".format(path, n, *box))
    sp.run(["packmol"], cwd=tmpdir, check=True, capture_output=True)
    return tmpdir / "packed.xyz"


def build_msi(outfile: Path, mols: List[Chem.Mol], counts: List[int], packed_xyz: Path, box: Tuple[float, float, float]):
    """Write Biosym MDF w/ all mandatory blocks"""
    # read packed xyz coordinates
    coords = []
    with packed_xyz.open() as fh:
        next(fh); next(fh)  # skip header
        for line in fh:
            sym, x, y, z = line.split()
            coords.append((sym, float(x), float(y), float(z)))

    total_atoms = sum(len(mol.GetAtoms()) * n for mol, n in zip(mols, counts))
    assert total_atoms == len(coords), "coord / topology mismatch"

    with outfile.open("w") as out:
        out.write("!BIOSYM molecular_data 3\n")
        # System & Box
        out.write("@system\n        temperature 300.0\n@periodicbox\n 0 0 0   {:g} 0 0   0 {:g} 0   0 0 {:g}\n".format(*box))

        # Substructure table
        out.write("@substructure\n")
        sub_id = 1
        for mol, n in zip(mols, counts):
            for i in range(n):
                out.write(f"{sub_id} MOL{sub_id}\n")
                sub_id += 1

        # Atom table
        out.write("@atom\n")
        atom_index = 1
        coord_idx = 0
        sub_id = 1
        for mol, n in zip(mols, counts):
            for i in range(n):
                for atom in mol.GetAtoms():
                    sym, x, y, z = coords[coord_idx]
                    coord_idx += 1
                    elem = atom.GetSymbol()
                    typ = atom_type_from_elem(elem)
                    charge = float(atom.GetProp('_GasteigerCharge')) if atom.HasProp('_GasteigerCharge') else 0.0
                    name = f"{elem}{atom_index}"
                    out.write(f"{name:<12s} {typ:<3s} {typ:<3s} ? 0 0 {charge:8.4f} 0 0 8 1.0000 0.0000 {x:8.3f} {y:8.3f} {z:8.3f} {sub_id}\n")
                    atom_index += 1
                sub_id += 1

        # Bond table (single bonds by RDKit perception)
        out.write("@bond\n")
        atom_offset = 0
        sub_id = 1
        for mol, n in zip(mols, counts):
            for k in range(n):
                for b in mol.GetBonds():
                    i = atom_offset + b.GetBeginAtomIdx() + 1
                    j = atom_offset + b.GetEndAtomIdx() + 1
                    order = b.GetBondTypeAsDouble()
                    out.write(f"{i} {j} {int(order)}\n")
                atom_offset += mol.GetNumAtoms()
                sub_id += 1
        out.write("end\n")

###############################################################################
# CLI
###############################################################################

def parse():
    p = argparse.ArgumentParser(description="Build MSI/MDF for msi2lmp")
    p.add_argument('--smiles', action='append', required=True, help='SMILES string (repeatable)')
    p.add_argument('--count',  action='append', type=int, required=True, help='Count for each SMILES (repeatable)')
    p.add_argument('--box', nargs=3, type=float, required=True, metavar=('LX','LY','LZ'), help='Box lengths in Å')
    p.add_argument('-o', '--outfile', default='system.msi')
    return p.parse_args()


def main():
    args = parse()
    if len(args.smiles) != len(args.count):
        raise ValueError('number of --smiles and --count must match')

    mols = [generate_3d(smi) for smi in args.smiles]
    box = tuple(args.box)

    with TemporaryDirectory() as td:
        tmpdir = Path(td)
        packed_xyz = run_packmol(mols, args.count, box, tmpdir)
        build_msi(Path(args.outfile), mols, args.count, packed_xyz, box)
        print(f"✓ Wrote {args.outfile} – ready for msi2lmp")

if __name__ == '__main__':
    main()


usage: ipykernel_launcher.py [-h] --smiles SMILES --count COUNT --box LX LY LZ
                             [-o OUTFILE]
ipykernel_launcher.py: error: the following arguments are required: --smiles, --count, --box


SystemExit: 2